In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터셋 불러오기
data = pd.read_csv("Cleaned_Laptop_data.csv")

In [3]:
# 불러온 데이터 확인
"""
<모든 데이터>
index = index 추가함
brand = 노트북 브랜드
model = 모델명
processor_brand = cpu 브랜드
processor_name = 코어명 (i5. i3 ...etc)
processor_gnrtn = cpu 세대
ram_gb = 램 용량
ram_type = 램 세대
ssd = ssd 용량 / 없으면 '0'
hdd = hdd 용량 / 없으면 '0'
os = os 종류
os_bit = os 비트(64/ 32)
graphic_card_gb = 그래픽 카드 용량
weight = 무게 thin/casual/heavy
display_size = 화면 인치 (missing 및 다양함)
warranty = 보증기간
touchscreen = 화면 터치 가능/ 불가능(yes or no)
msoffice = 마소 깔려있는지 없는지 (yes or no)
latest_price = 최신가격
old_price = 출고가
discount = 할인률
star_rating = 평점 평균
review = 평점 카운트
rating = 인기도
-----------------------------------
<사용하고자 하는 데이터>
processor_name = 코어명 (i5. i3 ...etc)
ram_gb = 램 용량
ssd = ssd 용량 / 없으면 '0'
weight = 무게 thin/casual/heavy
latest_price = 최신가격
star_rating = 평점 평균
reviews = 평점 카운트
ratings = 인기도

"""
data.head(2)

,index,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
0,0,Lenovo,A6-9225,AMD,A6-9225 Processor,10th,4 GB,DDR4,0 GB,1024 GB,...,Missing,0,No,No,24990,32790,23,3.7,63,12
1,1,Lenovo,Ideapad,AMD,APU Dual,10th,4 GB,DDR4,0 GB,512 GB,...,Missing,0,No,No,19590,21325,8,3.6,1894,256


In [4]:
# 데이터 형태 확인
data.shape

(896, 24)

In [5]:
empty_df = pd.DataFrame()

In [6]:
empty_df["index"] = data["index"]
empty_df["brand_model"] = data["brand"] + "-" + data["model"]
empty_df["cpu_spec"] = data["processor_brand"]+" "+data["processor_name"]+"-"+data["processor_gnrtn"]
empty_df["ram_spec"] = data["ram_gb"] + " " + data["ram_type"]
empty_df["storage"] = data["ssd"] + " " + "SSD" + " " + data["hdd"] + " " + "HDD"
empty_df["computer_specific_function"] = data["weight"] + " " + data["display_size"] + " " + data["Touchscreen"]
empty_df["price"] = data["latest_price"].apply(str) # TfidfVector로 만들기 위해서 int를 str으로 만듬

In [7]:
empty_df.head(20)

,index,brand_model,cpu_spec,ram_spec,storage,computer_specific_function,price
0,0,Lenovo-A6-9225,AMD A6-9225 Processor-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,ThinNlight Missing No,24990
1,1,Lenovo-Ideapad,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,19590
2,2,Avita-PURA,AMD APU Dual-10th,4 GB DDR4,128 GB SSD 0 GB HDD,ThinNlight Missing No,19990
3,3,Avita-PURA,AMD APU Dual-10th,4 GB DDR4,128 GB SSD 0 GB HDD,ThinNlight Missing No,21490
4,4,Avita-,AMD APU Dual-10th,4 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight Missing No,24990
5,5,Avita-PURA,AMD APU Dual-10th,8 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight 14 No,24990
6,6,HP-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual 14 No,20900
7,7,Lenovo-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,21896
8,8,HP-Athlon,AMD Athlon Dual-10th,32 GB DDR4,32 GB SSD 0 GB HDD,Casual 14 No,26899
9,9,acer-Aspire,AMD Athlon Dual-10th,4 GB DDR4,256 GB SSD 0 GB HDD,Casual Missing No,31990


In [8]:
empty_df.shape

(896, 7)

In [9]:
tfidf_vector = TfidfVectorizer() #TfidfVector변환 함수

In [10]:
#모든 columnT에 대해서 TfidfVector matrix로 변환
tfidf_matrix_all = tfidf_vector.fit_transform(empty_df['brand_model'] + " " + empty_df['cpu_spec'] + " " + empty_df['ram_spec'] + " " + empty_df['storage'] + " " + empty_df['computer_specific_function'] + " " + empty_df['price']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()

#brand_model의 dataframe를 만듬
tfidf_matrix_all = pd.DataFrame(tfidf_matrix_all, columns=tfidf_matrix_feature, index = empty_df.brand_model)
print(tfidf_matrix_all.shape)
tfidf_matrix_all.head()


(896, 608)


,100190,100990,101990,1024,102551,102990,103990,104000,104990,105990,...,wf65,windows,x1,x360,x390,xps,yes,yoga,zenbook,zephyrus
brand_model,,,,,,,,,,,,,,,,,,,,,
Lenovo-A6-9225,0.0,0.0,0.0,0.094224,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lenovo-Ideapad,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Avita-PURA,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Avita-PURA,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Avita-,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 유사도 구하기

In [12]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix_all)

Wall time: 26.6 ms


In [13]:
cosine_sim.shape

(896, 896)

In [14]:
#brand_model기준 cosine_similarity dataframe 만듬
cosine_sim_df = pd.DataFrame(cosine_sim, index = empty_df.brand_model, columns = empty_df.brand_model)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(896, 896)


brand_model,Lenovo-A6-9225,Lenovo-Ideapad,Avita-PURA,Avita-PURA,Avita-,Avita-PURA,HP-APU,Lenovo-APU,HP-Athlon,acer-Aspire,...,ASUS-VivoBook,ASUS-VivoBook,ASUS-Ryzen,ASUS-ROG,ASUS-ROG,ASUS-ROG,ASUS-Ryzen,ASUS-Ryzen,SAMSUNG-Galaxy,Lenovo-Thinkpad
brand_model,,,,,,,,,,,,,,,,,,,,,
Lenovo-A6-9225,1.000000,0.122440,0.092811,0.092811,0.242103,0.203800,0.082541,0.111645,0.050471,0.089045,...,0.111663,0.123337,0.107645,0.100960,0.110615,0.107869,0.122435,0.110373,0.040442,0.139329
Lenovo-Ideapad,0.122440,1.000000,0.418958,0.418958,0.510667,0.431496,0.561389,0.621118,0.169707,0.288428,...,0.189132,0.210682,0.182327,0.170750,0.187357,0.182707,0.209142,0.186947,0.097535,0.248519
Avita-PURA,0.092811,0.418958,1.000000,0.793635,0.583468,0.697703,0.459235,0.474928,0.134871,0.229777,...,0.143872,0.161600,0.138695,0.129697,0.142521,0.138984,0.160419,0.142210,0.063397,0.173073
Avita-PURA,0.092811,0.418958,0.793635,1.000000,0.583468,0.697703,0.459235,0.474928,0.134871,0.229777,...,0.143872,0.161600,0.138695,0.129697,0.142521,0.138984,0.160419,0.142210,0.063397,0.173073
Avita-,0.242103,0.510667,0.583468,0.583468,1.000000,0.857375,0.559760,0.578888,0.164394,0.318955,...,0.175365,0.196974,0.169055,0.158087,0.173719,0.169407,0.195534,0.173339,0.077275,0.210958


In [15]:
# 컴퓨터 이름으로 추천해줌
def computer_title_recommendations(target_title, matrix, items, k=5):
    #타겟 타이틀이 가지고 있는 인덱스를 뽑음
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    #타겟 타이틀에 대한 cos similarity가 높은 순으로 index의 values를 출력
    recom_title = items.iloc[recom_idx, :].brand_model.values
    recom_cpu_spec = items.iloc[recom_idx, :].cpu_spec.values
    recom_ram_spec = items.iloc[recom_idx, :].ram_spec.values
    recom_storage = items.iloc[recom_idx, :].storage.values
    recom_specific_function = items.iloc[recom_idx, :].computer_specific_function.values
    recome_price = items.iloc[recom_idx, :].price.values
    #찾는 타이틀 검색
    target_title_list = np.full(len(range(k)), target_title)
    
    d = {
        'target_title':target_title_list,
        'recom_title' : recom_title,
        'recom_cpu_spec' : recom_cpu_spec,
        'recom_ram_spec' : recom_ram_spec,
        'recom_storage_spec' : recom_storage,
        'recom_function' : recom_specific_function,
        'price': recome_price
    }
    return pd.DataFrame(d)

In [16]:
computer_title_recommendations('Avita-PURA', cosine_sim_df, empty_df)

,target_title,recom_title,recom_cpu_spec,recom_ram_spec,recom_storage_spec,recom_function,price
0,Avita-PURA,HP-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual 14 No,20900
1,Avita-PURA,ASUS-ExpertBook,Intel Core i3-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual 15.6 No,34649
2,Avita-PURA,ASUS-VivoBook,Intel Celeron Dual-Missing,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,26990
3,Avita-PURA,acer-Aspire,AMD Athlon Dual-10th,4 GB DDR4,256 GB SSD 0 GB HDD,Casual Missing No,31990
4,Avita-PURA,Lenovo-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,21896


In [17]:
computer_title_recommendations('HP-APU', cosine_sim_df, empty_df)

,target_title,recom_title,recom_cpu_spec,recom_ram_spec,recom_storage_spec,recom_function,price
0,HP-APU,Lenovo-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,21896
1,HP-APU,Lenovo-Ideapad,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,19590
2,HP-APU,Avita-,AMD APU Dual-10th,4 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight Missing No,24990
3,HP-APU,Avita-PURA,AMD APU Dual-10th,8 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight 14 No,24990
4,HP-APU,ASUS-APU,AMD Quad-10th,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,19990


In [18]:
computer_title_recommendations('Lenovo-APU', cosine_sim_df, empty_df)

,target_title,recom_title,recom_cpu_spec,recom_ram_spec,recom_storage_spec,recom_function,price
0,Lenovo-APU,HP-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual 14 No,20900
1,Lenovo-APU,Lenovo-Ideapad,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,19590
2,Lenovo-APU,Avita-,AMD APU Dual-10th,4 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight Missing No,24990
3,Lenovo-APU,ASUS-APU,AMD Quad-10th,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,19990
4,Lenovo-APU,Avita-PURA,AMD APU Dual-10th,8 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight 14 No,24990


In [19]:
#price기준 cosine_similarity dataframe 만듬
cosine_sim_df_price = pd.DataFrame(cosine_sim, index = empty_df.brand_model, columns = empty_df.price)
print(cosine_sim_df_price.shape)
cosine_sim_df_price.head()

(896, 896)


price,24990,19590,19990,21490,24990,24990,20900,21896,26899,31990,...,130500,116990,138990,234990,135990,144990,149990,142990,38990,57490
brand_model,,,,,,,,,,,,,,,,,,,,,
Lenovo-A6-9225,1.000000,0.122440,0.092811,0.092811,0.242103,0.203800,0.082541,0.111645,0.050471,0.089045,...,0.111663,0.123337,0.107645,0.100960,0.110615,0.107869,0.122435,0.110373,0.040442,0.139329
Lenovo-Ideapad,0.122440,1.000000,0.418958,0.418958,0.510667,0.431496,0.561389,0.621118,0.169707,0.288428,...,0.189132,0.210682,0.182327,0.170750,0.187357,0.182707,0.209142,0.186947,0.097535,0.248519
Avita-PURA,0.092811,0.418958,1.000000,0.793635,0.583468,0.697703,0.459235,0.474928,0.134871,0.229777,...,0.143872,0.161600,0.138695,0.129697,0.142521,0.138984,0.160419,0.142210,0.063397,0.173073
Avita-PURA,0.092811,0.418958,0.793635,1.000000,0.583468,0.697703,0.459235,0.474928,0.134871,0.229777,...,0.143872,0.161600,0.138695,0.129697,0.142521,0.138984,0.160419,0.142210,0.063397,0.173073
Avita-,0.242103,0.510667,0.583468,0.583468,1.000000,0.857375,0.559760,0.578888,0.164394,0.318955,...,0.175365,0.196974,0.169055,0.158087,0.173719,0.169407,0.195534,0.173339,0.077275,0.210958


In [20]:
# 예산으로 추천해줌
def computer_price_recommendations(target_price, matrix, items, k=5):
    #타겟 price이 가지고 있는 인덱스를 뽑음
    recom_idx = matrix.loc[:, target_price].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    #타겟 price에 대한 cos similarity가 높은 순으로 index의 values를 출력
    recom_title = items.iloc[recom_idx, :].brand_model.values
    recom_cpu_spec = items.iloc[recom_idx, :].cpu_spec.values
    recom_ram_spec = items.iloc[recom_idx, :].ram_spec.values
    recom_storage = items.iloc[recom_idx, :].storage.values
    recom_specific_function = items.iloc[recom_idx, :].computer_specific_function.values
    recome_price = items.iloc[recom_idx, :].price.values
    #찾는 타이틀 검색
    target_title_list = np.full(len(range(k)), target_price)
    
    d = {
        'target_title':target_title_list,
        'recom_title' : recom_title,
        'recom_cpu_spec' : recom_cpu_spec,
        'recom_ram_spec' : recom_ram_spec,
        'recom_storage_spec' : recom_storage,
        'recom_function' : recom_specific_function,
        'price': recome_price
    }
    return pd.DataFrame(d)

In [21]:
computer_price_recommendations('19590', cosine_sim_df_price, empty_df)

,target_title,recom_title,recom_cpu_spec,recom_ram_spec,recom_storage_spec,recom_function,price
0,19590,Lenovo-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,21896
1,19590,HP-APU,AMD APU Dual-10th,4 GB DDR4,0 GB SSD 1024 GB HDD,Casual 14 No,20900
2,19590,Avita-,AMD APU Dual-10th,4 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight Missing No,24990
3,19590,ASUS-APU,AMD Quad-10th,4 GB DDR4,0 GB SSD 512 GB HDD,Casual Missing No,19990
4,19590,Avita-PURA,AMD APU Dual-10th,8 GB DDR4,256 GB SSD 0 GB HDD,ThinNlight 14 No,24990


In [22]:
computer_price_recommendations('34649', cosine_sim_df_price, empty_df)

,target_title,recom_title,recom_cpu_spec,recom_ram_spec,recom_storage_spec,recom_function,price
0,34649,ASUS-ExpertBook,Intel Core i3-10th,4 GB DDR4,256 GB SSD 1024 GB HDD,Casual Missing No,40990
1,34649,ASUS-ExpertBook,Intel Core i5-10th,8 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,56990
2,34649,ASUS-ExpertBook,Intel Core i3-11th,4 GB DDR4,256 GB SSD 0 GB HDD,Casual Missing No,35990
3,34649,ASUS-ExpertBook,Intel Core i5-10th,8 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,61249
4,34649,ASUS-ExpertBook,Intel Core i5-10th,8 GB DDR4,0 GB SSD 1024 GB HDD,Casual Missing No,83990


In [23]:
computer_price_recommendations('130500', cosine_sim_df_price, empty_df)

,target_title,recom_title,recom_cpu_spec,recom_ram_spec,recom_storage_spec,recom_function,price
0,130500,ASUS-VivoBook,AMD Ryzen 7-Missing,4 GB DDR4,1024 GB SSD 0 GB HDD,Casual 16 No,99990
1,130500,ASUS-VivoBook,AMD Ryzen 9-Missing,4 GB DDR4,1024 GB SSD 0 GB HDD,Casual 16 No,104990
2,130500,ASUS-VivoBook,AMD Ryzen 9-Missing,4 GB DDR4,1024 GB SSD 0 GB HDD,Casual 16 No,162990
3,130500,ASUS-VivoBook,AMD Ryzen 7-Missing,4 GB DDR4,512 GB SSD 0 GB HDD,Casual 16 No,56990
4,130500,ASUS-VivoBook,AMD Ryzen 5-Missing,8 GB DDR4,256 GB SSD 1024 GB HDD,Casual 15.6 No,54990
